**Lab performance 02**

**Name:TANJIDUR RAHMAN TUHIN**

**ID :0242310005341018**


In [7]:
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

Saving Breast Cancer Wisconsin (Diagnostic) Data Set.csv to Breast Cancer Wisconsin (Diagnostic) Data Set (1).csv
Saving Pima Indians Diabetes Dataset.csv to Pima Indians Diabetes Dataset.csv
Saving Titanic - Machine Learning from Disaster.csv to Titanic - Machine Learning from Disaster.csv
Saving UCI Heart Disease Data.csv to UCI Heart Disease Data.csv
Saving Wine Dataset.csv to Wine Dataset.csv
Saving Wine Dataset.xlsx to Wine Dataset.xlsx
User uploaded file "Breast Cancer Wisconsin (Diagnostic) Data Set (1).csv" with length 125141 bytes
User uploaded file "Pima Indians Diabetes Dataset.csv" with length 23873 bytes
User uploaded file "Titanic - Machine Learning from Disaster.csv" with length 28629 bytes
User uploaded file "UCI Heart Disease Data.csv" with length 79346 bytes
User uploaded file "Wine Dataset.csv" with length 390376 bytes
User uploaded file "Wine Dataset.xlsx" with length 389354 bytes


In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
df = pd.read_csv('UCI Heart Disease Data.csv')
print("--- Data Head ---")
print(df.head())
print("\n--- Data Info ---")
print(df.info())

--- Data Head ---
   id  age     sex    dataset               cp  trestbps   chol    fbs  \
0   1   63    Male  Cleveland   typical angina     145.0  233.0   True   
1   2   67    Male  Cleveland     asymptomatic     160.0  286.0  False   
2   3   67    Male  Cleveland     asymptomatic     120.0  229.0  False   
3   4   37    Male  Cleveland      non-anginal     130.0  250.0  False   
4   5   41  Female  Cleveland  atypical angina     130.0  204.0  False   

          restecg  thalch  exang  oldpeak        slope   ca  \
0  lv hypertrophy   150.0  False      2.3  downsloping  0.0   
1  lv hypertrophy   108.0   True      1.5         flat  3.0   
2  lv hypertrophy   129.0   True      2.6         flat  2.0   
3          normal   187.0  False      3.5  downsloping  0.0   
4  lv hypertrophy   172.0  False      1.4    upsloping  0.0   

                thal  num  
0       fixed defect    0  
1             normal    2  
2  reversable defect    1  
3             normal    0  
4             norm

In [16]:
df = df.replace('?', np.nan)
df = df.dropna()
df['ca'] = pd.to_numeric(df['ca'])
df = df.drop(columns=['id', 'dataset'], errors='ignore')

In [21]:
if 'num' in df.columns:
    df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)
    df = df.drop(columns=['num'])

In [22]:
categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
cols_to_encode = [col for col in categorical_cols if col in df.columns and df[col].dtype == 'object']
df_encoded = pd.get_dummies(df, columns=cols_to_encode, drop_first=True)
print("\n--- Encoded Data Head ---")
print(df_encoded.head())


--- Encoded Data Head ---
   age  trestbps   chol  thalch  oldpeak   ca  target  sex_Male  \
0   63     145.0  233.0   150.0      2.3  0.0       0      True   
1   67     160.0  286.0   108.0      1.5  3.0       1      True   
2   67     120.0  229.0   129.0      2.6  2.0       1      True   
3   37     130.0  250.0   187.0      3.5  0.0       0      True   
4   41     130.0  204.0   172.0      1.4  0.0       0     False   

   cp_atypical angina  cp_non-anginal  cp_typical angina  fbs_True  \
0               False           False               True      True   
1               False           False              False     False   
2               False           False              False     False   
3               False            True              False     False   
4                True           False              False     False   

   restecg_normal  restecg_st-t abnormality  exang_True  slope_flat  \
0           False                     False       False       False   
1      

In [24]:
X = df_encoded.drop('target', axis=1)
y = df_encoded['target']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nTraining set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")


Training set size: 239
Testing set size: 60


In [26]:
numeric_cols = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak']
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])
print("\n--- Scaled Training Data Head ---")
print(X_train.head())


--- Scaled Training Data Head ---
          age  trestbps      chol    thalch   oldpeak   ca  sex_Male  \
68   0.457310  2.105261  1.426142 -0.406643  2.069449  0.0      True   
208  0.012108 -0.126283  0.241228  0.252704 -0.913230  0.0      True   
178 -1.323498 -0.126283  1.222485  0.560399  0.753561  1.0      True   
134 -1.323498 -0.572592 -0.665972  0.692269 -0.737778  0.0     False   
189  1.570315  0.431603  0.093114 -0.142905  0.841287  3.0      True   

     cp_atypical angina  cp_non-anginal  cp_typical angina  fbs_True  \
68                False           False              False     False   
208                True           False              False     False   
178               False            True              False     False   
134               False            True              False     False   
189               False            True              False     False   

     restecg_normal  restecg_st-t abnormality  exang_True  slope_flat  \
68            False       

In [29]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
conf_matrix_rf = confusion_matrix(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)

print("\n==============================================")
print("       Random Forest Model Results            ")
print("==============================================")
print(f"Accuracy: {accuracy_rf:.4f}\n")
print("Confusion Matrix:\n", conf_matrix_rf)
print("\nClassification Report:\n", report_rf)


       Random Forest Model Results            
Accuracy: 0.8167

Confusion Matrix:
 [[29  3]
 [ 8 20]]

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.91      0.84        32
           1       0.87      0.71      0.78        28

    accuracy                           0.82        60
   macro avg       0.83      0.81      0.81        60
weighted avg       0.82      0.82      0.81        60

